In [6]:
import csv
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('./data') if isfile(join('./data', f))]

In [17]:
import re

In [70]:
text = list()
labels = list()
for file in onlyfiles:    
    with open('data/' + file, newline='', encoding="utf8") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        if 'trump' not in file.lower():
            label = 'NoTrump'
        else:
            label = 'Trump'
        text_ind = 1
        if "MORE_TRUMP.csv" in file:
            text_ind = 0
        for row in csv_reader:            
            if line_count != 0:                
                line = re.sub('[\s\n\t#@]+', ' ', row[text_ind])
                line = re.sub(r'http\S+', '', line).strip()            
                text.append(line)
                labels.append(label)            
            line_count += 1
                

In [58]:
from collections import Counter
c = Counter(labels)
c

Counter({'NoTrump': 23178, 'Trump': 12531})

In [78]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = \
train_test_split(text, labels, test_size=0.33, random_state=42)

In [99]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()
# vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=3, max_df=200)
trainX = vectorizer.fit_transform(train_data)
testX = vectorizer.transform(test_data)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_labels)
trainy = le.transform(train_labels)
testy = le.transform(test_labels)

In [100]:
from sklearn.linear_model import LogisticRegression
cls = LogisticRegression(random_state=0, solver='lbfgs', max_iter=10000)
cls.fit(trainX, trainy)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [101]:
def evaluate(X, yt, cls, name='data'):
	"""Evaluated a classifier on the given labeled data using accuracy."""
	from sklearn import metrics
	yp = cls.predict(X)
	acc = metrics.accuracy_score(yt, yp)
	print("  Accuracy on %s  is: %s" % (name, acc))

In [102]:
evaluate(testX, testy, cls)

  Accuracy on data  is: 0.9176001357773251
